In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
import os
n_jobs = 16
os.environ["OMP_NUM_THREADS"] = str(n_jobs)
import joblib
import click
import json
import time
from glob import glob
import itertools
import collections.abc
import sys
from tqdm.notebook import tqdm
# !{sys.executable} -m pip install qcircuit
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import re
from joblib import Parallel, delayed, dump, load
import datetime

In [2]:
df_model = pd.concat([pd.read_csv(i,index_col=0).dropna() for i in sorted(glob('*csv'))])

/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_14408/4208722250.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_model = pd.concat([pd.read_csv(i,index_col=0).dropna() for i in sorted(glob('*csv'))])


In [3]:
df_model['Cost'].shape

(426,)

In [4]:
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12
 
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [5]:
top=os.getcwd()
with open(os.path.join(top,'0.1_5_DDCC_train.bin'),'rb') as f:
    ddcc_train = joblib.load(f)

with open(os.path.join(top,'0.1_5_DDCC_test.bin'),'rb') as f:
    ddcc_test = joblib.load(f)

with open(os.path.join(top,'0.1_5_DDCC_scaler.bin'),'rb') as f:
    ddcc_scaler = joblib.load(f)

X_ddcc_train, y_ddcc_train = ddcc_train['X'],ddcc_train['y']
X_ddcc_test, y_ddcc_test = ddcc_test['X'],ddcc_test['y']

X_ddcc_train = X_ddcc_train.reshape(-1,64,5)
X_ddcc_test = X_ddcc_test.reshape(-1,64,5)
y_ddcc_train = y_ddcc_train.reshape(-1,64)
y_ddcc_test = y_ddcc_test.reshape(-1,64)


# X_train, y_train = X_ddcc_train, y_ddcc_train
# X_test, y_test = X_ddcc_test, y_ddcc_test
X_train, y_train = X_ddcc_train, y_ddcc_train
X_test, y_test = X_ddcc_test, y_ddcc_test

# X_train = [X_train[i:i+4] for i in range(0,len(X_train),4)]
# X_test = [X_test[i:i+4] for i in range(0,len(X_test),4)]
scaler = ddcc_scaler

# print(len(X_train),X_train[0].shape,X_train[-1].shape)
print(y_train.shape, y_test.shape)

(19, 64) (40, 64)


/Users/grierjones/miniconda3/envs/simple_qml/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
def grab_data(path):
    '''
    Given a globbed path, return the job_n.txt, *_train.txt, and *_test.txt files
    '''
    iterations = []
    train_metric = []
    test_metric = []
    for i in path:
        try:
            numbers = int(re.search(r'jobs_(\d+)\.txt', i).group(1))
            iterations.append(i)
        except:
            if 'test' in i:
                test_metric.append(i)
            else:
                train_metric.append(i)
            
    return sorted(iterations,key=lambda x: int(re.search(r'jobs_(\d+)\.txt', x).group(1))), train_metric, test_metric

In [7]:
def fetch_jobs_from_file(service, filename):
    """Fetch all job results from a given file."""
    with open(filename, 'r') as f:
        job_ids = f.readlines()
    return [service.job(job_id.strip()).result() for job_id in job_ids]

def grab_jobs(lst):
    """Parallelize over job files, fetching results concurrently."""


    service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance='pinq-quebec-hub/univ-toronto/pr-hans-arno-jac'
    )
    # Parallel processing of files
    jobs = Parallel(n_jobs=-1,backend='threading')(
        delayed(fetch_jobs_from_file)(service, filename) for filename in tqdm(lst)
    )
    
    return np.array(jobs)

In [8]:
def fetch(job):
    try:
        return np.array([list(i.data.values()) for i in service.job(job).result()]).flatten()
    except:
        return job

In [9]:
default_data=pd.concat([pd.read_csv(i) for i in glob('default_partition/workloads_*/*csv')]).sort_values(by='Created')

In [10]:
founddata = pd.concat([default_data[default_data['JobId']==i] for i in np.hstack([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])])

/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_14408/1861948771.py:1: UserWarning: genfromtxt: Empty input file: "run5/jobs_63.txt"
  founddata = pd.concat([default_data[default_data['JobId']==i] for i in np.hstack([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])])


# 74 iter left, grab the length of each iter run

In [11]:
len([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])

/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_14408/1675272610.py:1: UserWarning: genfromtxt: Empty input file: "run5/jobs_63.txt"
  len([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])


394

In [12]:
checkdf = default_data.set_index('JobId').loc[[np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')][0]]


/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_14408/3394434878.py:1: UserWarning: genfromtxt: Empty input file: "run5/jobs_63.txt"
  checkdf = default_data.set_index('JobId').loc[[np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')][0]]


In [18]:
jobsall = []
for i in glob('run*/*.txt'):
    jobsdf = []
    for j in np.genfromtxt(i,dtype=str):
        try:
            print(j,default_data.set_index('JobId')['Usage (seconds)'].loc[j])
            jobsdf.append(default_data.set_index('JobId')['Usage (seconds)'].loc[j])
        except:
            # print(i)
            continue
    print(i,jobsdf)
    jobsall.append(np.sum(jobsdf))

d0dzky76rr3g00884h60 70
d0dzky7d8drg008zj8yg 70
d0dzkyfqnmvg0087yfk0 83
d0dzkyq6rr3g00884h6g 70
d0dzkyq6rr3g00884h70 77
d0dzkz7kzhn0008w6k0g 70
d0dzm0gqnmvg0087yfkg 70
d0dzm0rrxz8g008b28z0 77
d0dzm18nhqag008qcybg 70
d0dzm18nhqag008qcyc0 70
d0dzm1rqnmvg0087yfm0 77
d0dzm2gkzhn0008w6k10 70
d0dzm3gnhqag008qcydg 70
d0dzm3gd8drg008zj8zg 70
d0dzm3r6rr3g00884h8g 70
d0dzm40rxz8g008b290g 83
d0dzm50rxz8g008b2920 70
d0dzm50d8drg008zj900 70
d0dzm58kzhn0008w6k1g 70
run2/jobs_7.txt [70, 70, 83, 70, 77, 70, 70, 77, 70, 70, 77, 70, 70, 70, 70, 83, 70, 70, 70]
d0e3njyqnmvg0087zhwg 71
d0e3nk6kzhn0008w7pc0 70
d0e3nk66rr3g00885k10 77
d0e3nk6kzhn0008w7pbg 77
d0e3nkenhqag008qe0m0 70
d0e3nkpqnmvg0087zhx0 70
d0e3npeqnmvg0087zhy0 77
d0e3npp6rr3g00885k20 70
d0e3npp6rr3g00885k30 70
d0e3npp6rr3g00885k1g 71
d0e3nppkzhn0008w7pcg 70
d0e3ns7qnmvg0087zhz0 70
d0e3nszkzhn0008w7pgg 70
d0e3nsznhqag008qe0n0 70
d0e3nt7rxz8g008b3afg 70
d0e3nt7d8drg008zkarg 70
d0e3ntfkzhn0008w7ph0 70
d0e3nv7rxz8g008b3agg 70
d0e3nvfqnmvg0087zj0

/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_14408/1573850988.py:4: UserWarning: genfromtxt: Empty input file: "run5/jobs_63.txt"
  for j in np.genfromtxt(i,dtype=str):


run5/jobs_18.txt []
run5/jobs_24.txt []
run5/jobs_30.txt []
run5/jobs_31.txt []
run5/jobs_25.txt []
run5/jobs_19.txt []
run5/jobs_8.txt []
run5/jobs_21.txt []
run5/jobs_35.txt []
run5/jobs_34.txt []
run5/jobs_20.txt []
run5/jobs_9.txt []
run5/jobs_36.txt []
run5/jobs_22.txt []
run5/jobs_23.txt []
run5/jobs_37.txt []
run5/jobs_50.txt []
run5/jobs_44.txt []
run5/jobs_45.txt []
run5/jobs_51.txt []
run5/jobs_47.txt []
run5/jobs_53.txt []
run5/jobs_52.txt []
run5/jobs_46.txt []
run5/jobs_42.txt []
run5/jobs_56.txt []
run5/jobs_57.txt []
run5/jobs_43.txt []
run5/jobs_55.txt []
run5/jobs_41.txt []
run5/jobs_40.txt []
run5/jobs_54.txt []
run4/jobs_7.txt []
run4/jobs_12.txt []
run4/jobs_13.txt []
run4/jobs_6.txt []
run4/jobs_4.txt []
run4/jobs_11.txt []
run4/jobs_10.txt []
run4/jobs_5.txt []
run4/jobs_1.txt []
run4/jobs_14.txt []
run4/jobs_15.txt []
run4/jobs_0.txt []
run4/jobs_2.txt []
run4/jobs_17.txt []
run4/jobs_16.txt []
run4/jobs_3.txt []
run4/jobs_18.txt []
run4/jobs_19.txt []
run4/jobs_

In [23]:
timings_per_iter = np.array(jobsall)
np.mean(timings_per_iter[timings_per_iter!=0])

1351.0618556701031

In [29]:
(74 * 1351.0618556701031 ) /  60 / 60

27.771827033218788

In [31]:
(1351.0618556701031 / 60) / 4

5.62942439862543

In [54]:
(70*4 * 74) / 60 / 60

5.7555555555555555

In [40]:
 checkdf['Usage (seconds)'].mean()

72.47368421052632

In [19]:
[default_data[default_data['JobId']==j].shape for j in np.hstack([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])]

/var/folders/px/qyhp9b8j4bb0z5wgydzgkpxw0000gn/T/ipykernel_11744/345042408.py:1: UserWarning: genfromtxt: Empty input file: "run5/jobs_63.txt"
  [default_data[default_data['JobId']==j].shape for j in np.hstack([np.genfromtxt(i,dtype=str) for i in glob('run*/*.txt')])]


[(1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),


In [11]:
founddata

,JobId,SessionId,Status,Reason,Created,Completed,Usage (seconds),Type,Compute resource,Instance,Tags
3,d0dzky76rr3g00884h60,d0dzktqqnmvg0087yfg0,Completed,NaN,2025-05-08T00:25:28.682Z,2025-05-08T00:54:46.615Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
3,d0dzky7d8drg008zj8yg,d0dzktqnhqag008qcy7g,Completed,NaN,2025-05-08T00:25:28.531Z,2025-05-08T01:00:07.508Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
3,d0dzkyfqnmvg0087yfk0,d0dzktqrxz8g008b28y0,Completed,NaN,2025-05-08T00:25:29.919Z,2025-05-08T01:05:59.900Z,83,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
3,d0dzkyq6rr3g00884h6g,d0dzktqkzhn0008w6jz0,Completed,NaN,2025-05-08T00:25:30.517Z,2025-05-08T01:11:35.366Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
2,d0dzkyq6rr3g00884h70,d0dzktqnhqag008qcy8g,Completed,NaN,2025-05-08T00:25:30.649Z,2025-05-08T01:17:16.527Z,77,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
...,...,...,...,...,...,...,...,...,...,...,...
0,d0dnpdsd8drg008zfbz0,d0dnp20rxz8g008aza5g,Failed,Internal error -- https://ibm.biz/error_codes#...,2025-05-07T13:08:07.394Z,2025-05-07T13:46:28.656Z,9,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
0,d0dnpdsd8drg008zfbyg,d0dnp20kzhn0008w3jv0,Completed,NaN,2025-05-07T13:08:07.286Z,2025-05-07T13:44:06.742Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
0,d0dnpds6rr3g00881h60,d0dnp20nhqag008q9xt0,Completed,NaN,2025-05-07T13:08:07.779Z,2025-05-07T13:57:10.291Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN
1,d0dnpdsqnmvg0087vh00,d0dnp20rxz8g008aza50,Completed,NaN,2025-05-07T13:08:07.702Z,2025-05-07T13:52:06.936Z,70,estimator,ibm_quebec,pinq-quebec-hub/univ-toronto/default,NaN


In [ ]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/univ-toronto/default'
)

data =  Parallel(n_jobs=-1)(
        delayed(fetch)(i) for i in tqdm(founddata['JobId'])
    )
 

In [ ]:
y_ddcc_train.shape

In [ ]:
192 / 64

In [ ]:
5502 / 19

In [ ]:
np.vstack([i for i in data if type(i) is not str]).flatten().shape

In [ ]:
# # three_months_ago = datetime.datetime.now() - datetime.timedelta(days=21)
 
# service = QiskitRuntimeService(
#     channel='ibm_quantum',
#     instance='pinq-quebec-hub/univ-toronto/default'
# )
# jobs_in_last_months = service.jobs(created_after=datetime.datetime(2025,5,6),descending=True,instance='pinq-quebec-hub/univ-toronto/default',limit=None)



In [ ]:
# data_recent =  Parallel(n_jobs=-1)(
#         delayed(fetch)(i.job_id()) for i in tqdm(jobs_in_last_months) if i.primitive_id == 'estimator'
#     )

In [ ]:

datadelta = datetime.datetime.now() - datetime.datetime(2025,5,6)
 
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/univ-toronto/pr-hans-arno-jac'
)
jobs_in_last_months = service.jobs(created_after=datadelta,descending=True,instance='pinq-quebec-hub/univ-toronto/pr-hans-arno-jac',limit=None)

data_recent =  Parallel(n_jobs=-1)(
        delayed(fetch)(i.job_id()) for i in tqdm(jobs_in_last_months) if i.primitive_id == 'estimator'
    )

In [ ]:
len(jobs_in_last_months)

In [ ]:
data_recent =  Parallel(n_jobs=-1)(
        delayed(fetch)(i.job_id()) for i in tqdm(jobs_in_last_months) if i.primitive_id == 'estimator'
    )

In [ ]:
jobs_in_last_months[0].primitive_id

In [ ]:
jobs_in_last_months[0].job_id()

In [ ]:
help(service.jobs)

In [ ]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/univ-toronto/default'
)
service.job('d0crshyd8drg008zaby0')

In [ ]:
help(service.jobs)

In [ ]:
three_months_ago

In [ ]:
jobs_in_last_months

In [ ]:
jobids = []
for i in sorted(glob("run*/*txt"),key=lambda x: (x.split('/')[0],int(x.split('/')[1].replace('jobs_','').replace('.txt','')))):
    with open(i,'r') as f:
        jobids.append([i.strip() for i in f.readlines()])

In [ ]:


[i.job_id() for i in jobs_in_last_months if i.job_id() in sum(jobids,[])]

In [ ]:
data = []
for i in sorted(glob('run*')):
    if os.path.isdir(i):
        print(i)
        try:
            data.append(grab_jobs(sorted(glob(os.path.join(i,'job*txt')),key=lambda x: int(x.split('/')[1].replace('jobs_','').replace('.txt','')))))
        except:
            print(f"This one is cooked {i}")

In [ ]:
data = grab_jobs(sorted(glob("run*/*txt"),key=lambda x: (x.split('/')[0],int(x.split('/')[1].replace('jobs_','').replace('.txt',''))))) 

In [ ]:
# results_10 = grab_jobs(iter_10)
# y_10 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_10]).reshape(*results_10.shape)

# with open('y_10.npy', 'wb') as f:
#     np.save(f, y_10)


# results_train_10 = grab_jobs(train_10)
# y_10_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_10]).reshape(*y_train.shape)

# results_test_10 = grab_jobs(test_10)
# y_10_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_10]).reshape(*y_test.shape)

# with open('y_10_test.npy', 'wb') as f:
#     np.save(f, y_10_test)
# with open('y_10_train.npy', 'wb') as f:
#     np.save(f, y_10_train)
    
# results_100 = grab_jobs(iter_100)
# y_100 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_100]).reshape(*results_100.shape)

# results_500 = grab_jobs(iter_500)
# y_500 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_500]).reshape(*results_500.shape)

# results_1000 = grab_jobs(iter_1000)
# y_1000 = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_1000]).reshape(*results_1000.shape)
# with open('y_100.npy', 'wb') as f:
#     np.save(f, y_100)
# with open('y_500.npy', 'wb') as f:
#     np.save(f, y_500)
# with open('y_1000.npy', 'wb') as f:
#     np.save(f, y_1000)

# results_train_100 = grab_jobs(train_100)
# y_100_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_100]).reshape(*y_train.shape)


# results_train_500 = grab_jobs(train_500)
# y_500_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_500]).reshape(*y_train.shape)

# results_train_1000 = grab_jobs(train_1000)
# y_1000_train = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_train_1000]).reshape(*y_train.shape)

# results_test_100 = grab_jobs(test_100)
# y_100_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_100]).reshape(*y_test.shape)


# results_test_500 = grab_jobs(test_500)
# y_500_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_500]).reshape(*y_test.shape)

# results_test_1000 = grab_jobs(test_1000)
# y_1000_test = np.array([[[k.data.evs.flatten() for k in j] for j in i] for i in results_test_1000]).reshape(*y_test.shape)

    
# with open('y_100_test.npy', 'wb') as f:
#     np.save(f, y_100_test)
# with open('y_500_test.npy', 'wb') as f:
#     np.save(f, y_500_test)
# with open('y_1000_test.npy', 'wb') as f:
#     np.save(f, y_1000_test)


# with open('y_100_train.npy', 'wb') as f:
#     np.save(f, y_100_train)
# with open('y_500_train.npy', 'wb') as f:
#     np.save(f, y_500_train)
# with open('y_1000_train.npy', 'wb') as f:
#     np.save(f, y_1000_train)

In [ ]:
with open('y_10_test.npy', 'rb') as f:
    y_10_test = np.load(f)

with open('y_100_test.npy', 'rb') as f:
    y_100_test = np.load(f)
with open('y_500_test.npy', 'rb') as f:
    y_500_test = np.load(f)
with open('y_1000_test.npy', 'rb') as f:
    y_1000_test = np.load(f)   


with open('y_10_train.npy', 'rb') as f:
    y_10_train = np.load(f)    
with open('y_100_train.npy', 'rb') as f:
    y_100_train = np.load(f)
with open('y_500_train.npy', 'rb') as f:
    y_500_train = np.load(f)
with open('y_1000_train.npy', 'rb') as f:
    y_1000_train = np.load(f)       


with open('y_10.npy', 'rb') as f:
    y_10 = np.load(f)
with open('y_100.npy', 'rb') as f:
    y_100 = np.load(f)
with open('y_500.npy', 'rb') as f:
    y_500 = np.load(f)
with open('y_1000.npy', 'rb') as f:
    y_1000 = np.load(f)    
    


In [ ]:
y_10_test = scaler.inverse_transform(y_10_test.flatten().reshape(-1,1)).flatten()
y_100_test = scaler.inverse_transform(y_100_test.flatten().reshape(-1,1)).flatten()
y_500_test = scaler.inverse_transform(y_500_test.flatten().reshape(-1,1)).flatten()
y_1000_test = scaler.inverse_transform(y_1000_test.flatten().reshape(-1,1)).flatten()

y_10_train = scaler.inverse_transform(y_10_train.flatten().reshape(-1,1)).flatten()
y_100_train = scaler.inverse_transform(y_100_train.flatten().reshape(-1,1)).flatten()
y_500_train = scaler.inverse_transform(y_500_train.flatten().reshape(-1,1)).flatten()
y_1000_train = scaler.inverse_transform(y_1000_train.flatten().reshape(-1,1)).flatten()

In [ ]:
y_10 = scaler.inverse_transform(y_10.flatten().reshape(-1,1)).reshape(*y_10.shape)
y_100 = scaler.inverse_transform(y_100.flatten().reshape(-1,1)).reshape(*y_100.shape)
y_500 = scaler.inverse_transform(y_500.flatten().reshape(-1,1)).reshape(*y_500.shape)
y_1000 = scaler.inverse_transform(y_1000.flatten().reshape(-1,1)).reshape(*y_1000.shape)



y_ddcc_train = scaler.inverse_transform(y_ddcc_train.flatten().reshape(-1,1)).reshape(*y_ddcc_train.shape)
y_ddcc_test = scaler.inverse_transform(y_ddcc_test.flatten().reshape(-1,1)).reshape(*y_ddcc_test.shape)

In [ ]:
r2_score(y_ddcc_train.flatten(),y_10_train.flatten())

In [ ]:
r2_score(y_ddcc_train.flatten(),y_100_train.flatten())

In [ ]:
r2_score(y_ddcc_train.flatten(),y_500_train.flatten())

In [ ]:
r2_score(y_ddcc_train.flatten(),y_1000_train.flatten())

In [ ]:
MSE_10 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_10]
MSE_100 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_100]
MSE_500 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_500]
MSE_1000 = [mean_squared_error(y_ddcc_train.flatten(),i.flatten()) for i in y_1000]

combined_MSE = MSE_10+MSE_100+MSE_500+MSE_1000
min_MSE_idx = np.argmin(MSE_10+MSE_100+MSE_500+MSE_1000)

In [ ]:
best_train = np.vstack([y_10,y_100,y_500,y_1000])[min_MSE_idx]

In [ ]:
with open(os.path.join(os.path.expanduser('~'),'qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_results.json')) as f:
    statevector = json.load(f)

In [ ]:
'/Users/grierjones/qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_predicted_values.csv'

In [ ]:
statevectordf = pd.read_csv(os.path.join(os.path.expanduser('~'),'qregress/qml_DDCC/RUD_AL/5AL/A2_HWE-CNOT/A2_HWE-CNOT_predicted_values.csv'))
statevectordf['Predicted'] = [float(i.strip('[]')) for i in statevectordf['Predicted']]
statevectordf['Reference'] = [float(i.strip('[]')) for i in statevectordf['Reference']]
statevectordf['Device'] = len(statevectordf)*['State Vector']



In [ ]:
statevectordf

In [ ]:
dfbest = pd.DataFrame(np.vstack([y_ddcc_train.flatten(),best_train.flatten()]),index=['Reference','Predicted']).T
dfbest['Device'] = len(dfbest)*['ibm_quebec']
dfbest['Data'] = len(dfbest)*['Train']

In [ ]:
spread = 10e-2
# Create the figure with a 2D grid (scatter + KDE for Predicted + KDE for Reference)
fig, axes = plt.subplots(
    2, 2, 
    figsize=(8, 8),
    gridspec_kw={'width_ratios': [4, 1], 'height_ratios': [1, 4]},
    constrained_layout=True,
    # sharey='row',  # Keep y-axis sharing, but remove sharex to control ticks manually,
    # sharex='col'
)
axes[0, -1].axis("off")
sns.scatterplot(data=dfbest,x='Reference',y='Predicted',label="Best ibm_quebec: R$^{2}=$"+f"{r2_score(dfbest['Reference'],dfbest['Predicted']):.4f}", edgecolors='black',ax=axes[1,0])
# sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector: R$^{2}=$"+f"{r2_score(statevectordf['Reference'],statevectordf['Predicted']):.4f}", edgecolors='black',ax=axes[1,0])
axes[1,0].plot(range(-1,2),range(-1,2),'k--')
axes[1,0].set_ylim(-spread,spread)
axes[1,0].set_xlim(-spread,spread)
axes[1,0].set_ylabel("Predicted t$_{2}$-amplitudes")
axes[1,0].set_xlabel("Calculated t$_{2}$-amplitudes")
axes[1,0].legend()


# sns.kdeplot(data=pd.concat([dfbest,statevectordf[statevectordf['Data']=='Train']]),hue='Device',x='Reference',ax=axes[0,0],fill=True)
sns.kdeplot(data=dfbest,x='Reference',ax=axes[0,0],fill=True, bw_adjust=2)
# sns.kdeplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',ax=axes[0,0],fill=True, bw_adjust=2)
axes[0,0].set_xticklabels([])  # Hide labels but keep ticks
axes[0,0].set_xlabel("")  # Remove x-labels
axes[0,0].set_xlim(-spread,spread)
axes[0,0].set_ylim(0,100)

# sns.kdeplot(data=pd.concat([dfbest,statevectordf[statevectordf['Data']=='Train']]),hue='Device',y='Predicted',ax=axes[1,1],fill=True)
sns.kdeplot(data=dfbest,y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2)
# sns.kdeplot(data=statevectordf[statevectordf['Data']=='Train'],y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2)
axes[1,1].set_yticklabels([])  # Hide labels but keep ticks
axes[1,1].set_ylabel("")  # Remove x-labels
axes[1,1].set_ylim(-spread,spread)
axes[1,1].set_xlim(0,100)
# plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust width and height spacing
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/bestibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
# # plt.scatter(y_ddcc_train.flatten(),best_train.flatten(),label="Best ibm_quebec R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),best_train.flatten()):.4f}",color='r', edgecolors='black')
# sns.scatterplot(data=dfbest,x='Reference',y='Predicted',label="Best ibm_quebec: R$^{2}=$"+f"{r2_score(dfbest['Reference'],dfbest['Predicted']):.4f}", edgecolors='black')
# sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector: R$^{2}=$"+f"{r2_score(statevectordf['Reference'],statevectordf['Predicted']):.4f}", edgecolors='black')
# plt.plot(range(-1,2),range(-1,2),'k--')
# plt.ylim(-3e-2,3e-2)
# plt.xlim(-3e-2,3e-2)
# plt.ylabel("Predicted t$_{2}$-amplitudes")
# plt.xlabel("Calculated t$_{2}$-amplitudes")
# plt.legend()
# plt.tight_layout()
# plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/bestibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
# plt.show()

In [ ]:
plt.scatter(y_ddcc_train.flatten(),y_1000_train.flatten(),label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_train.flatten()):.4f}",color='b', edgecolors='black')
plt.scatter(y_ddcc_test.flatten(),y_1000_test.flatten(),label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_test.flatten()):.4f}",color='g', edgecolors='black')
sns.scatterplot(data=statevectordf[statevectordf['Data']=='Train'],x='Reference',y='Predicted',label="State Vector Train: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data']=='Train']['Reference'],statevectordf[statevectordf['Data']=='Train']['Predicted']):.4f}", edgecolors='black')
sns.scatterplot(data=statevectordf[statevectordf['Data']=='Test'],x='Reference',y='Predicted',label="State Vector Test: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data']=='Test']['Reference'],statevectordf[statevectordf['Data']=='Test']['Predicted']):.4f}", edgecolors='black')
plt.plot(range(-1,2),range(-1,2),'k--')
plt.ylim(-3e-2,3e-2)
plt.xlim(-3e-2,3e-2)
plt.ylabel("Predicted t$_{2}$-amplitudes")
plt.xlabel("Calculated t$_{2}$-amplitudes")
plt.legend()
plt.tight_layout()
# plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/finalibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
finaldf = pd.DataFrame(np.hstack([np.vstack([y_ddcc_train.flatten(),y_1000_train.flatten()]),np.vstack([y_ddcc_test.flatten(),y_1000_test.flatten()])]),index=['Reference','Predicted']).T
finaldf['Data'] = len(y_ddcc_train.flatten())*['Train']+len(y_ddcc_test.flatten())*['Test']
finaldf['Device'] = len(finaldf)*['ibm_quebec']

In [ ]:
y_ddcc_train.shape

In [ ]:
cmap = sns.color_palette('Paired',6)

In [ ]:
cmap

In [ ]:
finaldf["Data"] = finaldf["Data"] + " (" + finaldf["Device"] + ")"
statevectordf["Data"] = statevectordf["Data"] + " (" + statevectordf["Device"] + ")"


In [ ]:
statevectordf['Reference']

In [ ]:
finaldf

In [ ]:
y_1000_train.flatten()

In [ ]:
spread = 10e-2
# Create the figure with a 2D grid (scatter + KDE for Predicted + KDE for Reference)
fig, axes = plt.subplots(
    2, 2, 
    figsize=(10, 10),
    gridspec_kw={'width_ratios': [4, 1.7], 'height_ratios': [1.7, 4]},
    constrained_layout=True,
    # sharey='row',  # Keep y-axis sharing, but remove sharex to control ticks manually,
    # sharex='col'
)
axes[0, -1].axis("off")
# sns.scatterplot(data=finaldf,x='Reference',y='Predicted',hue='Data',style='Data',markers=['o', 's'], edgecolors='black',ax=axes[1,0],palette=)
axes[1,0].scatter(y_ddcc_train.flatten(),y_1000_train.flatten(),marker='s',label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_train.flatten()):.4f}",color=cmap[0], edgecolors='black')
axes[1,0].scatter(y_ddcc_test.flatten(),y_1000_test.flatten(),label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(),y_1000_test.flatten()):.4f}",color=cmap[1], edgecolors='black')
# sns.scatterplot(data=statevectordf,hue='Data',x='Reference',y='Predicted',style='Data',markers=['d', 'D'], edgecolors='black',ax=axes[1,0],palette=cmap[2:4])
axes[1,0].plot(range(-1,2),range(-1,2),'k--')
axes[1,0].set_ylim(-spread,spread)
axes[1,0].set_xlim(-spread,spread)
axes[1,0].set_ylabel("Predicted t$_{2}$-amplitudes")
axes[1,0].set_xlabel("Calculated t$_{2}$-amplitudes")
axes[1,0].legend()


# sns.histplot(data=pd.concat([finaldf,statevectordf]),hue='Data',x='Reference',ax=axes[0,0],fill=True,palette=cmap[0:4])
# sns.histplot(data=finaldf,hue='Data',x='Reference',ax=axes[0,0],fill=True,palette=cmap[0:2],stat='probability',kde=True)
# axes[0,0].set_yscale('log')
sns.kdeplot(data=finaldf,hue='Data',x='Reference',ax=axes[0,0],fill=True, bw_adjust=2,palette=cmap[0:2])
# sns.kdeplot(data=statevectordf,hue='Data',x='Reference',ax=axes[0,0],fill=True, bw_adjust=2,palette=cmap[3:5])
axes[0,0].set_xticklabels([])  # Hide labels but keep ticks
axes[0,0].set_xlabel("")  # Remove x-labels
axes[0,0].set_xlim(-spread,spread)
axes[0,0].set_ylim(1,60)
# axes[0,0].legend(loc=3)

# sns.histplot(data=pd.concat([finaldf,statevectordf]),hue='Data',y='Predicted',ax=axes[1,1],fill=True,palette=cmap[0:4])
# sns.histplot(data=finaldf,hue='Data',y='Predicted',ax=axes[1,1],fill=True,palette=cmap[0:2],stat='probability',kde=True)
# axes[1,1].set_xscale('log')
sns.kdeplot(data=finaldf,hue='Data',y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2,palette=cmap[0:2])
# sns.kdeplot(data=statevectordf,hue='Data',y='Predicted',ax=axes[1,1],fill=True, bw_adjust=2,palette=cmap[3:5])
# axes[1,1].set_xticks(np.hstack([0,np.logspace(0,4,3)]))
# axes[1,1].set_xticklabels(['0']+["10$^{"+f"{np.log10(i):n}"+"}$" for i in np.logspace(0,4,3)])  # Hide labels but keep ticks
axes[1,1].set_yticklabels([])  # Hide labels but keep ticks
axes[1,1].set_ylabel("")  # Remove x-labels
axes[1,1].set_ylim(-spread,spread)
axes[1,1].set_xlim(1,60)
plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust width and height spacing
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/finalibm_vs_statevector.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
len([len(y_ddcc_train.flatten())*['Train']]+[len(y_ddcc_test.flatten())*['Test']])

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from sklearn.metrics import r2_score

# fig, ax = plt.subplots(figsize=(8, 6))

# # Scatter plots
# ax.scatter(y_ddcc_train.flatten(), y_1000_train.flatten(), 
#            label="ibm_quebec Train R$^{2}=$"+f"{r2_score(y_ddcc_train.flatten(), y_1000_train.flatten()):.4f}", 
#            color='b', edgecolors='black')
# ax.scatter(y_ddcc_test.flatten(), y_1000_test.flatten(), 
#            label="ibm_quebec Test R$^{2}=$"+f"{r2_score(y_ddcc_test.flatten(), y_1000_test.flatten()):.4f}", 
#            color='g', edgecolors='black')

# sns.scatterplot(data=statevectordf[statevectordf['Data'] == 'Train'], x='Reference', y='Predicted', 
#                 label="State Vector Train: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data'] == 'Train']['Reference'], statevectordf[statevectordf['Data'] == 'Train']['Predicted']):.4f}", 
#                 ax=ax, edgecolor='black')

# sns.scatterplot(data=statevectordf[statevectordf['Data'] == 'Test'], x='Reference', y='Predicted', 
#                 label="State Vector Test: R$^{2}=$"+f"{r2_score(statevectordf[statevectordf['Data'] == 'Test']['Reference'], statevectordf[statevectordf['Data'] == 'Test']['Predicted']):.4f}", 
#                 ax=ax, edgecolor='black')

# # Identity line
# ax.plot(np.linspace(-1, 2, 100), np.linspace(-1, 2, 100), 'k--')

# ax.set_ylim(-3e-2, 3e-2)
# ax.set_xlim(-3e-2, 3e-2)
# ax.set_ylabel("Predicted t$_{2}$-amplitudes")
# ax.set_xlabel("Calculated t$_{2}$-amplitudes")
# ax.legend()
# plt.tight_layout()

# # Create twin axes for KDE plots
# top_ax = ax.twiny()
# right_ax = ax.twinx()

# # KDE distributions
# sns.kdeplot(statevectordf['Reference'], ax=top_ax, color='gray', lw=2, clip=(-3e-2, 3e-2))
# sns.kdeplot(statevectordf['Predicted'], ax=right_ax, color='gray', lw=2, clip=(-3e-2, 3e-2))

# # Hide tick labels for KDE axes
# top_ax.set_xticks([])
# right_ax.set_yticks([])

# plt.show()


In [ ]:
len(MSE_10+MSE_100+MSE_500+MSE_1000)

In [ ]:
plt.scatter(min_MSE_idx,combined_MSE[min_MSE_idx],color='r',label=f'MSE:{combined_MSE[min_MSE_idx]:.4e}')
plt.plot(range(1,len(MSE_10+MSE_100+MSE_500+MSE_1000)+1),MSE_10+MSE_100+MSE_500+MSE_1000)
plt.ylabel('Training Loss (MSE)')
plt.xlabel('Iterations')
plt.xlim(0,900)
plt.ylim(0,8e-4)
# plt.hlines(statevector['MSE_train'][0],-100,1e4,color='r',linestyle='--',label=f'State Vector MSE:{statevector['MSE_train'][0]:.4e}')
# plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(os.path.expanduser('~'),'qregress/images/DDCC/ibmq_loss.png'),dpi=300,bbox_inches='tight')
plt.show()